In [11]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import BatchNormalization
#from tensorflow.keras.layers import Conv2D
#from tensorflow.keras.layers import MaxPooling2D
#from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense
#from tensorflow.keras import backend as K
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [12]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256,256))
image_size = 0
directory_root = '../input/plantvillage-dataset'
width=256
height=256
depth=3


In [13]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, (64,64))
            img = img_to_array(image)
            #img=img.flatten()
            #print(img.shape)
            return img
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None


In [14]:

image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir[:1] :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        print(f"{directory_root}/{plant_folder}")

        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")

            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")
except Exception as e:
    print(f"Error : {e}")



[INFO] Loading images ...
../input/plantvillage-dataset/color
[INFO] Processing Tomato___Tomato_mosaic_virus ...
[INFO] Processing Strawberry___healthy ...
[INFO] Processing Cherry_(including_sour)___Powdery_mildew ...
[INFO] Processing Corn_(maize)___healthy ...
[INFO] Processing Grape___Esca_(Black_Measles) ...
[INFO] Processing Apple___Black_rot ...
[INFO] Processing Orange___Haunglongbing_(Citrus_greening) ...
[INFO] Processing Raspberry___healthy ...
[INFO] Processing Tomato___Target_Spot ...
[INFO] Processing Tomato___Bacterial_spot ...
[INFO] Processing Peach___healthy ...
[INFO] Processing Corn_(maize)___Northern_Leaf_Blight ...
[INFO] Processing Pepper,_bell___healthy ...
[INFO] Processing Peach___Bacterial_spot ...
[INFO] Processing Blueberry___healthy ...
[INFO] Processing Potato___Early_blight ...
[INFO] Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus ...
[INFO] Processing Tomato___Late_blight ...
[INFO] Processing Potato___healthy ...
[INFO] Processing Cherry_(including_

In [15]:
image_size = len(image_list)
print(len(image_list))


7552


In [16]:
image_labels = []

for label in label_list:
    if label.split('_')[len(label.split('_'))-1] == 'healthy':
        image_labels.append(0)
    else:
        image_labels.append(1)


In [17]:
np_image_list = np.array(image_list).reshape(np.array(image_list).shape[0], np.array(image_list).shape[1]*np.array(image_list).shape[2]*np.array(image_list).shape[3])
#image_labels = image_labels.flatten()
print(len(image_labels))
print(np_image_list.shape)


7552
(7552, 12288)


In [18]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)


[INFO] Spliting data to train, test


In [19]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [20]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(x_train, y_train)


SVC(random_state=0)

In [21]:
y_pred = classifier.predict(x_test)


In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))


[[ 301  152]
 [  42 1016]]
0.871608206485771


In [23]:
import pickle

filename = 'finalized_model.pkl'
pickle.dump(classifier, open(filename, 'wb'))